In [1]:
# Import the dependencies.
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import numpy as np
import re

# Import the CRSS connection string.
from config import crss_conn_string

ModuleNotFoundError: No module named 'config'

# CONNECT TO THE DATABASE

In [ ]:
# Create CSSR engine.
cssr_engine = create_engine(crss_conn_string);

In [ ]:
# Connect to server.
cssr_conn = cssr_engine.connect();

In [ ]:
# Create accidents, vehicles, people data frame.
avp_df = pd.read_sql("SELECT * FROM public.accident_vehicle_person", cssr_conn)

In [ ]:
# Show first 10 rows.
pd.set_option("display.max_columns", None)
avp_df.head()

In [ ]:
# Write to CSV
avp_df.to_csv('all_gas_no_brakes.csv')

# PREPROCCESSING

In [ ]:
# Describe Data
avp_df.info()

In [ ]:
# Recode Urban City : 0 = urban, 1 = city
avp_df['urbancity'] = avp_df['urbancity'].replace(1,0)
avp_df['urbancity'] = avp_df['urbancity'].replace(2,1)
avp_df['urbancity'].unique()

In [ ]:
# Recode Month : 1 = Jan, 12 = Dec
avp_df['month']= avp_df['month'].replace([1,2,3,4,5,6,7,8,9,10,11,12],['Jan','Feb','Mar','Apr','May',
                                                                             'Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
avp_df['month'].unique()

In [ ]:
# Recode Day of the week 1= Sunday, 7 = Saturday
avp_df['day_week']= avp_df['day_week'].replace([1,2,3,4,5,6,7],['Sun','Mon','Tues','Wed','Thus','Fri','Sat'])
avp_df['day_week'].unique()

In [ ]:
# Recode Alcohol - accident level 1= alcohol involved, 2,8,9 =  no alcohol
avp_df['alcohol'] = avp_df['alcohol'].replace([1,2,8,9],[1,0,0,0])
avp_df['alcohol'].unique()

In [ ]:
# Recode max severity - accident level 0,8,9 = none, 1,2,5 = minor, 3 = serious, 4,6 = fatal
avp_df['max_sev']=avp_df['max_sev'].replace([0,8,9,1,2,5,3,4,6],['none', 'none','none','minor','minor',
                                                                 'minor','serious','fatal','fatal'])
avp_df['max_sev'].unique()

In [ ]:
# Recode work zone 0 = no (0), 1,2,3,4 = yes (1)
avp_df['wrk_zone'] = avp_df['wrk_zone'].replace([0,1,2,3,4],[0,1,1,1,1])
avp_df['wrk_zone'].unique()

In [ ]:
# Recode lighting conditions 1= daylight, 2,3,6 = dark, 4=dawn 5=dusk, 7,8,9 = other
avp_df['lgt_cond']= avp_df['lgt_cond'].replace([1,2,3,6,4,5,7,8,9],['daylight','dark','dark','dark','dawn','dusk',
                                                                     'other','other','other'])
avp_df['lgt_cond'].unique()

In [ ]:
# Recode weather 1 = clear 2,3 = rain/sleet/hail, 4,11 = snow, 5 = fog/smoke, 6=windy, 7=blowing dirt, 
# 10=cloudy, 12=freezing rain, 8, 98, 99 = other
avp_df['weather']=avp_df['weather'].replace([1,2,3,4,11,5,6,7,10,12,8,98,99],['clear','rain_sleet','rain_sleet',
                                                                              'snow_blowsnow','snow_blowsnow',
                                                                              'fog_smoke','windy','blowing_dirt','cloudy',
                                                                              'freezing_rain','other','other','other'])
avp_df['weather'].unique()

In [ ]:
avp_df['m_harmname'].unique()

In [ ]:
# Recode most harmful event - vehicle level
avp_df['m_harmname'] = avp_df['m_harmname'].replace([
   'Motor Vehicle In-Transport',
    'Motor Vehicle in Motion Outside the Trafficway',
    'Working Motor Vehicle',
    
    'Parked Motor Vehicle', 
    
    'Pedalcyclist',
    'Pedestrian',
    'Non-Motorist on Personal Conveyance',
    'Ridden Animal or Animal Drawn Conveyance',
    'Live Animal',
    
    'Traffic Sign Support',
    'Utility Pole/Light Support',
    'Bridge Pier or Support',
    'Guardrail End',
    'Post, Pole or Other Supports',
    'Impact Attenuator/Crash Cushion',
    'Fire Hydrant',
    'Other Fixed Object',
    'Unknown Fixed Object',
    'Mail Box',
    'Traffic Signal Support',
    'Bridge Overhead Structure',
    'Building',
    
    'Bridge Rail (Includes parapet)',
    'Curb', 
    'Guardrail Face',
    'Concrete Traffic Barrier',
    'Other Traffic Barrier',
    'Wall', 
    'Cable Barrier',
    'Fence',
    
    'Shrubbery',
    'Tree (Standing Only)',
    'Boulder',
    'Snow Bank',
    
    'Embankment',
    'Ditch', 
    'Ground',
    'Culvert',
    
    'Fire/Explosion',
    
    'Motor Vehicle In-Transport Strikes or is Struck by Cargo, Persons or Objects Set-in-Motion from/by Another Motor Vehicle In Transport',
    'Object That Had Fallen From Motor Vehicle In-Transport',
    'Unknown Object Not Fixed',
    'Thrown or Falling Object',
    'Other Object (not fixed)',
    
    'Immersion or Partial Immersion',
    
    'Reported as Unknown',
    'Harmful Event, Details Not Reported',
    
    'Pavement Surface Irregularity (Ruts, Potholes, Grates, etc.)',
    'Jackknife (harmful to this vehicle)',
    'Other Non-Collision',
    'Cargo/Equipment Loss, Shift, or Damage [harmful]',
    'Rollover/Overturn',
    
    'Injured In Vehicle (Non-Collision)',
    'Fell/Jumped from Vehicle', 
    
     'Railway Vehicle',
    'Road Vehicle on Rails'    
],
[
    'harm_moving_veh',
    'harm_moving_veh',
    'harm_moving_veh',
    
    'harm_parked_veh',
    
    'harm_ped_animal',
    'harm_ped_animal',
    'harm_ped_animal',
    'harm_ped_animal',
    'harm_ped_animal',
    
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    'harm_fixed_manmade',
    
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    'harm_barrier',
    
    'harm_nat_object',
    'harm_nat_object',
    'harm_nat_object',
    'harm_nat_object',
    
    'harm_terrain',
    'harm_terrain',
    'harm_terrain',
    'harm_terrain',
    
    'harm_fire',
    
    'harm_object',
    'harm_object',
    'harm_object',
    'harm_object',
    'harm_object',
    
    'harm_water',
    
    'harm_unknown',
    'harm_unknown',
    
    'harm_lost_control',
    'harm_lost_control',
    'harm_lost_control',
    'harm_lost_control',
    'harm_lost_control',
    
    'harm_injury_fallout',
    'harm_injury_fallout',
    
    'harm_train',
    'harm_train'    
])
avp_df['m_harmname'].unique()

In [ ]:
avp_df['makename'].unique()

In [ ]:
# Recode vehicle make as make_country
avp_df['make_country'] = avp_df['makename'].replace([
    'Toyota',
    'Honda',
    'Subaru',
    'Nissan/Datsun',
    'Acura',
    'Suzuki',
    'Lexus',
    'Mazda', 
    'Mitsubishi', 
    'Infiniti', 
    'Isuzu',
    'Scion',
    
    'KIA',
    'Daewoo',
    'Hyundai',
    
    'Chevrolet', 
    'Ford',
    'Pontiac',
    'Cadillac',
    'Dodge', 
    'Chrysler', 
    'GMC',
    'Jeep / Kaiser-Jeep / Willys- Jeep', 
    'Buick / Opel',
    'Other Domestic Manufacturers', 
    'Lincoln',
    'Oldsmobile', 
    'Mercury',
    'Plymouth', 
    'Eagle', 
    'American Motors',
    'Saturn',
    
    'Freightliner', 
    'AM General', 
    'International Harvester/Navistar',
    'Peterbilt',
    'Mack',
    'Kenworth',
    'Thomas Built', 
    'Bluebird',
    'White/Autocar White/GMC',
    'Gillig', 
    'MCI',
    'Grumman',
    
    'Mercedes-Benz',
    'Volkswagen',
    'Audi',
    'BMW', 
    'Smart',
    'Porsche',
    
    'Victory',
    
    'Volvo',
    'Saab', 
    
    'Ducati', 
    'Harley-Davidson', 
    'Yamaha', 
    'Kawasaki',
    'Moto-Guzzi',
    
    'Jaguar',
    'Land Rover',
    'Triumph', 
    
    'Alfa Romeo',
    'Fiat',
    
    'Other Import',
    'Other Make',
    'Unknown Make',
    'Not Reported',
],
[
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    'make_Japan',
    
    'make_Korea',
    'make_Korea',
    'make_Korea',
    
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    'make_US',
    
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    'make_US_truck',
    
    'make_Germany',
    'make_Germany',
    'make_Germany',
    'make_Germany',
    'make_Germany',
    'make_Germany',
    
    'make_China',
    
    'make_Sweden',
    'make_Sweden',
    
    'make_motorcycle',
    'make_motorcycle',
    'make_motorcycle',
    'make_motorcycle',
    'make_motorcycle',
    
    'make_England',
    'make_England',
    'make_England',
    
    'make_Italy',
    'make_Italy',
    
    'make_other',
    'make_other',
    'make_other',
    'make_other',    
])
avp_df['make_country'].unique()

In [ ]:
# Recode tow_vehname - towing a vehichle 0 = no, 1 = yes
avp_df['tow_vehname']=avp_df['tow_vehname'].replace([
    'No Trailing Units',
    'One Trailing Unit',
    'Unknown',
    'Two Trailing Units',
    'Vehicle Towing Another Motor Vehicle - Non-Fixed Linkage',
    'Three or More Trailing Units',
    'Vehicle Towing Another Motor Vehicle - Fixed Linkage',
    'Yes, Number of Trailing Units Unknown'
],
[0,1,1,1,1,1,1,1])
avp_df['tow_vehname'].unique()

In [ ]:
# Determine median speed for imputing for unknown travel Speed
avp_df['trav_speed_temp'] = avp_df['trav_speed']
avp_df['trav_speed_temp'] = avp_df['trav_speed_temp'].replace([997,998,999],[np.NaN,np.NaN,np.NaN])
avp_df['trav_speed_temp'].describe()

In [ ]:
# Replace unknown travel speed with median travel speed 
avp_df['trav_speed']=avp_df['trav_speed'].replace([997,998,999],[155,23,23])
avp_df['trav_speed'].describe()

In [ ]:
# Recode deformedname
avp_df['deformedname']=avp_df['deformedname'].replace([
    'Not Reported',
    'Reported as Unknown'
],
[
    'Minor Damage',
    'Minor Damage'
])
avp_df['deformedname'].unique()

In [ ]:
# Recode towed Name 0 = no 1 = yes
avp_df['towedname']=avp_df['towedname'].replace([
    'Towed Due to Disabling Damage',
    'Not Towed',
    'Towed Not Due to Disabling Damage',
    'Towed, Unknown Reason',
    'Not Reported',
    'Towed But Not Due to Disabling Damage',
    'Reported as Unknown'
],[1,0,1,1,0,1,0])
avp_df['towedname'].unique()

In [ ]:
avp_df['speedrelname'].unique()

In [ ]:
# Recode speed related 0 = n 1 = y
avp_df['speedrelname']=avp_df['speedrelname'].replace([
    'No', 
    'Yes, Too Fast for Conditions',
    'Reported as Unknown',
    'Yes, Exceeded Speed Limit',
    'Yes, Specifics Unknown',
    'No Driver Present/Unknown if Driver Present', 
    'Yes, Racing'
],[0,1,0,1,1,0,1])
avp_df['speedrelname'].unique()

In [ ]:
avp_df['vtrafwayname'].unique()

In [ ]:
# Recode traffic way name 
avp_df['vtrafwayname']=avp_df['vtrafwayname'].replace([
    'Two-Way, Divided, Unprotected Median',
    'Not Reported',
    'Two-Way,  Divided, Positive  Median Barrier',
    'Two-Way, Not Divided',
    'Two-Way, Not Divided With a Continuous Left-Turn Lane',
    'Non-Trafficway or Driveway Access',
    'Entrance/Exit Ramp',
    'One-Way Trafficway', 
    'Reported as Unknown'
],
[
    'Two_way_div_med_nobar',
    'Two_way',
    'Two_way_div_med_bar',
    'Two_way',
    'Two-way',
    'Parking_lot_driveway',
    'Exit_on_ramp',
    'One-way',
    'Two-way'
])
avp_df['vtrafwayname'].unique()

In [ ]:
avp_df['vspd_lim'].value_counts()

In [ ]:
# Recode speed limits (impute with the mode = 45)
avp_df['vspd_lim']=avp_df['vspd_lim'].replace([98,99],[45,45])
avp_df['vspd_lim'].value_counts()

In [ ]:
avp_df['bdytyp_imname'].unique()

In [ ]:
# Recode body type
avp_df['bdytyp_imname']=avp_df['bdytyp_imname'].replace([
    '4-door sedan, hardtop',
    '5-door/4-door hatchback', 
    'Sedan/Hardtop, number of doors unknown', 
    'Hatchback, number of doors unknown',
    'Auto-based pickup (includes E1 Camino, Caballero, Ranchero, SSR, G8-ST, Subaru Brat, Rabbit Pickup)',
    'Large Limousine-more than four side doors or stretched chassis',
    'Auto-based panel (cargo station wagon, auto-based ambulance or hearse)',
    'Station Wagon (excluding van and truck based)',
    'Other or Unknown automobile type',
    'Unknown body type', 
    'Not Reported',
    
    '2-door sedan,hardtop,coupe',
    '3-door/2-door hatchback',
    '3-door coupe',
    
    'Convertible(excludes sun-roof,t-bar)',
    
    'Light Pickup',
    'Unknown light truck type', 
    'Unknown (pickup style) light conventional truck type',
    'Unknown light vehicle type (automobile,utility vehicle, van, or light truck)',
    'Other light conventional truck type',
    'Compact Utility (Utility Vehicle Categories Small and Midsize)', 
    'Cab Chassis Based (includes Rescue Vehicle, Light Stake, Dump, and Tow Truck)',
    
    'Minivan (Chrysler Town and Country, Caravan, Grand Caravan, Voyager, Voyager, Honda-Odyssey, ...)',
    'Large Van-Includes van-based buses (B150-B350, Sportsman, Royal Maxiwagon, Ram, Tradesman,...)',
    'Van-Based Bus GVWR greater than 10,000 lbs.', 
    'Unknown van type',
    'Step van (GVWR greater than 10,000 lbs.)',
    'Other van type (Hi-Cube Van, Kary)', 
    'Step-van or walk-in van (GVWR less than or equal to 10,000 lbs.)',
    
    'Large utility (ANSI D16.1 Utility Vehicle Categories and Full Size and Large)',
    'Medium/heavy Pickup (GVWR greater than 10,000 lbs.)',
    'Utility Vehicle, Unknown body type', 
    'Utility station wagon (includes suburban limousines, Suburban, Travellall, Grand Wagoneer)',
    'Unknown truck type (light/medium/heavy)',
    
    'Truck-tractor (Cab only, or with any number of trailing unit; any weight)',
    'Single-unit straight truck or Cab-Chassis (GVWR range 19,501 to 26,000 lbs.)',
    'Single-unit straight truck or Cab-Chassis (GVWR unknown)',
    'Unknown medium/heavy truck type', 
    'Single-unit straight truck or Cab-Chassis (GVWR range 10,001 to 19,500 lbs.)',
    'Single-unit straight truck or Cab-Chassis (GVWR greater than 26,000 lbs.)',
    'Unknown if single-unit or combination unit Medium Truck (GVWR range 10,001 lbs. to 26,000 lbs.)',
    'Unknown if single-unit or combination unit Heavy Truck (GVWR greater than 26,000 lbs.)',
    
    'Two Wheel Motorcycle (excluding motor scooters)',
    'Motor Scooter',
    'Moped or motorized bicycle',
    'Unknown motored cycle type',
    'Off-road Motorcycle', 
    'Unenclosed Three Wheel Motorcycle / Unenclosed Autocycle (1 Rear Wheel)',
    'Three-wheel Motorcycle (2 Rear Wheels)',
    'Other motored cycle type (mini-bikes, pocket motorcycles pocket bikes)',
    'Unknown Three Wheel Motorcycle Type',
    
    'School Bus',
    'Transit Bus (City Bus)',
    'Other Bus Type', 
    'Unknown Bus Type',
    'Cross Country/Intercity Bus',
    
    'Construction equipment other than trucks (includes graders)',  
    'Farm equipment other than trucks',
    
    'Medium/heavy truck based motorhome',
    'Medium/Heavy Vehicle Based Motor Home',
    'Camper or motorhome, unknown truck type',
    'Light Vehicle Based Motor Home (chassis mounted)',
    'Light Truck Based Motorhome (Chassis Mounted)',
    
    'Recreational Off-Highway Vehicle',
    'Other vehicle type (includes go-cart, fork-lift, city street sweeper dunes/swamp buggy)',
    'Low Speed Vehicle (LSV) / Neighborhood Electric Vehicle (NEV)',
    'ATV/ATC [All-Terrain Cycle]',
    'Golf Cart',
     
     'Snowmobile'
    
],[
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    '4_door_sedan',
    
    '2_door_sedan',
    '2_door_sedan',
    '2_door_sedan',
    
    'Convertable',
    
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    'Small_SUV_light_truck',
    
    'Van',
    'Van',
    'Van',
    'Van',
    'Van',
    'Van',
    'Van',
    
    'Large_SUV',
    'Large_SUV',
    'Large_SUV',
    'Large_SUV',
    'Large_SUV',
    
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    'Truck',
    
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    'Motorcylcle_trike',
    
    'Bus',
    'Bus',
    'Bus',
    'Bus',
    'Bus',
    
    'Construction_farm_equip',
    'Construction_farm_equip',
    
    'Motorhome_RV',
    'Motorhome_RV',
    'Motorhome_RV',
    'Motorhome_RV',
    'Motorhome_RV',
    
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    'ATV_rec_vehicle',
    
     'Snowmobile',
    
])
avp_df['bdytyp_imname'].unique()


In [ ]:
avp_df['p_crash1name'].unique()

In [ ]:
# Recode prior to crash movement: some not binned. unknowns binned with going straight
avp_df['p_crash1name'] = avp_df['p_crash1name'].replace([
    'Going Straight', 
    'Other(specify:)',
    'Unknown', 
    'No Driver Present / Unknown if Driver Present',
        
    'Decelerating in Road',
    'Stopped in Roadway',
    'Backing Up (other than for Parking Position)',
    
    'Starting in Road',
    'Accelerating in Road', 
    
    'Disabled or Parked in Travel lane', 
    'Entering a Parking Position',
    'Leaving a Parking Position',
     
]
,[
    'Going straight',
    'Going straight',
    'Going straight',
    'Going straight',
    
    'Stopping_backup',
    'Stopping_backup',
    'Stopping_backup',
    
    'Start on road',
    'Start on road',
    
    'Diasbled_parked',
    'Diasbled_parked',
    'Diasbled_parked',
])
avp_df['p_crash1name'].unique()

In [ ]:
# Recode restrictive device
avp_df['rest_usename'] = avp_df['rest_usename'].replace([
    'Shoulder and Lap Belt Used',
    'Shoulder Belt Only Used', 
    'Other',
    'Restraint Used - Type Unknown',
    'Lap Belt Only Used',
    'Reported as Unknown',
    'Not Reported',
    
    'None Used/Not Applicable', 

    'Child Restraint Type Unknown', 
    'Booster Seat', 
    'Child Restraint System - Forward Facing',
    'Child Restraint System  - Rear Facing',
    'Child Restraint System - Rear Facing',
    
    'Racing-Style Harness Used'
    
   
],[
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    'Seatbelt',
    
    'No_seatbelt',
    
    'Child_restraint',
    'Child_restraint',
    'Child_restraint',
    'Child_restraint',
    'Child_restraint',
    
    'Harness'
])

avp_df['rest_usename'].unique()

In [ ]:
# Recode retrain misuse 0 = no, 1 = yes
avp_df['rest_misname'] = avp_df['rest_misname'].replace([
    'No Indication of Mis-Use', 
    'None Used/Not Applicable',
    'Yes, Indication of Mis-Use'
],[0,0,1])
avp_df['rest_misname'].unique()

In [ ]:
# Recode helmet wear 0= no, 1=yes
avp_df['helm_usename']=avp_df['helm_usename'].replace([
    'Not Applicable',
    'Helmet, Other than DOT-Compliant Motorcycle Helmet',
    'Helmet, Unknown if DOT-Compliant', 
    'No Helmet', 
    'Not Reported',
    'DOT-Compliant Motorcycle Helmet',
    'Reported as Unknown if Helmet Worn'
],[0,1,1,0,0,1,0])
avp_df['helm_usename'].unique()

In [ ]:
# Recode helmet misuse 0 = no 1 = yes
avp_df['helm_misname']=avp_df['helm_misname'].replace([
    'None Used/Not Applicable', 
    'No Indication of Mis-Use',
    'Yes, Indication of Mis-Use'
],[0,0,1])
avp_df['helm_misname'].unique()

In [ ]:
# Recode police reported drinking 0 = no 1 = yes
avp_df['drinkingname'] = avp_df['drinkingname'].replace([
    'No (Alcohol Not Involved)', 
    'Not Reported', 
    'Reported as Unknown',
    'Yes (Alcohol Involved)'
],[0,0,0,1])
avp_df['drinkingname'].unique()

In [ ]:
avp_df['alc_resname'].unique()


In [ ]:
avp_df['alc_resname_08']=avp_df['alc_resname'].replace([
    
    'Test Not Given', 'Not Reported', 'Reported as Unknown if Tested',
       'AC Test Performed, Results Unknown', '0.220 % BAC', '0.000 % BAC',
       '0.070 % BAC', '0.176 % BAC', '0.200 % BAC', '0.080 % BAC',
       '0.244 % BAC', '0.170 % BAC', '0.020 % BAC', '0.125 % BAC',
       '0.216 % BAC', '0.140 % BAC', '0.090 % BAC', '0.190 % BAC',
       '0.230 % BAC', '0.150 % BAC',
       'Positive Reading with No Actual Value', '0.172 % BAC',
       '0.160 % BAC', '0.187 % BAC', '0.100 % BAC', '0.110 % BAC',
       '0.181 % BAC', '0.157 % BAC', '0.210 % BAC', '0.158 % BAC',
       '0.119 % BAC', '0.180 % BAC', '0.189 % BAC', '0.130 % BAC',
       '0.030 % BAC', '0.208 % BAC', '0.245 % BAC', '0.144 % BAC',
       '0.121 % BAC', '0.280 % BAC', '0.195 % BAC', '0.194 % BAC',
       '0.017 % BAC', '0.011 % BAC', '0.050 % BAC', '0.076 % BAC',
       '0.116 % BAC', '0.060 % BAC', '0.072 % BAC', '0.221 % BAC',
       '0.193 % BAC', '0.296 % BAC', '0.161 % BAC', '0.255 % BAC',
       '0.075 % BAC', '0.155 % BAC', '0.229 % BAC', '0.214 % BAC',
       '0.152 % BAC', '0.250 % BAC', '.94 % or Greater', '0.238 % BAC',
       '0.123 % BAC', '0.310 % BAC', '0.014 % BAC', '0.275 % BAC',
       '0.290 % BAC', '0.188 % BAC', '0.137 % BAC', '0.114 % BAC',
       '0.040 % BAC', '0.212 % BAC', '0.078 % BAC', '0.184 % BAC',
       '0.375 % BAC', '0.178 % BAC', '0.088 % BAC', '0.165 % BAC',
       '0.115 % BAC', '0.209 % BAC', '0.300 % BAC', '0.168 % BAC',
       '0.132 % BAC', '0.260 % BAC', '0.318 % BAC', '0.185 % BAC',
       '0.286 % BAC', '0.173 % BAC', '0.077 % BAC', '0.186 % BAC',
       '0.120 % BAC', '0.167 % BAC', '0.162 % BAC', '0.111 % BAC',
       '0.129 % BAC', '0.287 % BAC', '0.081 % BAC', '0.302 % BAC',
       '0.094 % BAC', '0.240 % BAC', '0.135 % BAC', '0.142 % BAC',
       '0.131 % BAC', '0.148 % BAC', '0.128 % BAC', '0.309 % BAC',
       '0.026 % BAC', '0.159 % BAC', '0.196 % BAC', '0.191 % BAC',
       '0.151 % BAC', '0.270 % BAC', '0.108 % BAC', '0.105 % BAC',
       '0.204 % BAC', '0.016 % BAC', '0.057 % BAC', '0.109 % BAC',
       '0.093 % BAC', '0.113 % BAC', '0.307 % BAC', '0.085 % BAC',
       '0.008 % BAC', '0.164 % BAC', '0.269 % BAC', '0.320 % BAC',
       '0.138 % BAC', '0.146 % BAC', '0.062 % BAC', '0.083 % BAC',
       '0.177 % BAC', '0.247 % BAC', '0.175 % BAC', '0.223 % BAC',
       '0.102 % BAC', '0.028 % BAC', '0.222 % BAC', '0.126 % BAC',
       '0.228 % BAC', '0.336 % BAC', '0.920 % BAC', '0.239 % BAC',
       '0.232 % BAC', '0.201 % BAC', '0.281 % BAC', '0.001 % BAC',
       '0.285 % BAC', '0.010 % BAC', '0.197 % BAC', '0.249 % BAC',
       '0.101 % BAC', '0.361 % BAC', '0.395 % BAC', '0.163 % BAC',
       '0.096 % BAC', '0.382 % BAC', '0.271 % BAC', '0.350 % BAC',
       '0.182 % BAC', '0.047 % BAC', '0.211 % BAC', '0.154 % BAC',
       '0.166 % BAC', '0.053 % BAC', '0.042 % BAC', '0.218 % BAC',
       '0.192 % BAC', '0.215 % BAC', '0.004 % BAC', '0.174 % BAC',
       '0.231 % BAC', '0.226 % BAC', '0.227 % BAC', '0.099 % BAC',
       '0.139 % BAC', '0.082 % BAC', '0.124 % BAC', '0.002 % BAC',
       '0.031 % BAC', '0.272 % BAC', '0.213 % BAC', '0.058 % BAC',
       '0.045 % BAC', '0.262 % BAC', '0.198 % BAC', '0.378 % BAC',
       '0.224 % BAC', '0.041 % BAC', '0.390 % BAC', '0.112 % BAC',
       '0.117 % BAC', '0.205 % BAC', '0.305 % BAC', '0.298 % BAC',
       '0.179 % BAC', '0.289 % BAC', '0.242 % BAC', '0.012 % BAC',
       '0.147 % BAC', '0.243 % BAC', '0.248 % BAC', '0.106 % BAC',
       '0.234 % BAC', '0.141 % BAC', '0.274 % BAC', '0.143 % BAC',
       '0.256 % BAC', '0.145 % BAC', '0.006 % BAC', '0.202 % BAC',
       '0.207 % BAC', '0.054 % BAC', '0.380 % BAC', '0.068 % BAC',
       '0.067 % BAC', '0.384 % BAC', '0.027 % BAC', '0.253 % BAC',
       '0.156 % BAC', '0.292 % BAC', '0.136 % BAC', '0.254 % BAC',
       '0.322 % BAC', '0.206 % BAC', '0.237 % BAC', '0.600 % BAC',
       '0.252 % BAC', '0.095 % BAC', '0.340 % BAC', '0.048 % BAC',
       '0.153 % BAC', '0.294 % BAC', '0.337 % BAC', '0.091 % BAC',
       '0.233 % BAC', '0.241 % BAC', '0.043 % BAC', '0.171 % BAC',
       '0.023 % BAC', '0.334 % BAC', '0.520 % BAC', '0.007 % BAC',
       '0.069 % BAC', '0.059 % BAC', '0.104 % BAC', '0.079 % BAC',
       '0.268 % BAC', '0.013 % BAC', '0.107 % BAC', '0.236 % BAC',
       '0.217 % BAC', '0.066 % BAC', '0.098 % BAC', '0.203 % BAC',
       '0.251 % BAC', '0.003 % BAC', '0.308 % BAC', '0.278 % BAC',
       '0.246 % BAC', '0.064 % BAC', '0.235 % BAC', '0.086 % BAC',
       '0.061 % BAC', '0.259 % BAC', '0.169 % BAC'
],[
   0,0,0,
    0,1,0,
    0,1,1,1,
    1,1,0,1,
    1,1,1,1,
    1,1,
    0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    0,1,1,1,
    1,1,1,1,
    0,0,0,0,
    1,0,0,1,
    1,1,1,1,
    0,1,1,1,
    1,1,1,1,
    1,1,0,1,
    1,1,1,1,
    0,1,0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    1,1,0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,1,
    0,1,1,1,
    1,1,1,1,
    1,0,0,1,
    1,1,1,1,
    0,1,1,1,
    1,1,0,1,
    1,1,1,1,
    1,0,1,1,
    1,1,1,1,
    1,1,1,0,
    1,0,1,1,
    1,1,1,1,
    1,1,1,1,
    1,0,1,1,
    1,0,0,1,
    1,1,0,1,
    1,1,1,1,
    1,1,1,0,
    0,1,1,0,
    0,1,1,1,
    1,0,1,1,
    1,1,1,1,
    1,1,1,0,
    1,1,1,1,
    1,1,1,1,
    1,1,0,1,
    1,0,1,0,
    0,1,0,1,
    1,1,1,1,
    1,1,1,1,
    1,1,1,0,
    1,1,1,1,
    1,1,0,1,
    0,1,1,0,
    0,0,1,0,
    1,0,1,1,
    1,0,1,1,
    1,0,1,1,
    1,0,1,1,
    0,1,1   
])
avp_df['alc_resname_08'].unique()

In [ ]:
# Recode drugs involved 0 = no, 1 = yes
avp_df['drugsname']=avp_df['drugsname'].replace([
    'No (drugs not involved)', 
    'Not Reported', 
    'Reported as Unknown',
    'Yes (drugs involved)'
],[0,0,0,1])
avp_df['drugsname'].unique()

In [ ]:
# recode transfer to hospital 0 = no 1 = yes
avp_df['hospitalname']=avp_df['hospitalname'].replace([
    'EMS Ground', 
    'Not Transported', 
    'Other', 
    'EMS Air',
    'EMS Unknown Mode', 
    'Reported as Unknown',
    'Transported  Unknown Source', 
    'Not Reported', 
    'Law Enforcement',
    'Not Transported for Treatment'
],[1,0,1,1,1,0,1,0,1,0])
avp_df['hospitalname'].unique()

In [ ]:
avp_df['locationname'].unique()
# NOT USEFUL

In [ ]:
avp_df['sex_imname'].value_counts()

In [ ]:
# Recode sex 0 = female, 1 = male : Assign not reported as male, unknown as female
avp_df['sex_imname']=avp_df['sex_imname'].replace([
    'Female', 
    'Male', 
    'Reported as Unknown', 
    'Not Reported'
],[0,1,0,1])

avp_df['sex_imname'].value_counts()

In [ ]:
# Recode person level of injury
avp_df['injsev_imname'] = avp_df['injsev_imname'].replace([
    'Suspected Serious Injury (A)', 
    'No Apparent Injury (O)',
    'Possible Injury (C)', 
    'Unknown/Not Reported',
    'Suspected Minor Injury (B)',
    'Fatal Injury (K)',
    'Injured, Severity Unknown', 
    'Died Prior to Crash*'
],[
    'serious',
    'none',
    'minor',
    'none',
    'minor',
    'fatal',
    'minor',
    'fatal'
 ])
avp_df['injsev_imname'].unique()

In [ ]:
# Recode personal alcohol use
avp_df['peralch_imname']=avp_df['peralch_imname'].replace([
    'No (Alcohol Not Involved)',
    'Yes (Alcohol Involved)'
],[0,1])
avp_df['peralch_imname'].unique()

In [ ]:
avp_df['seat_imname'].unique()

In [ ]:
avp_df['seat_imname']=avp_df['seat_imname'].replace([
    'Front Seat, Left Side', 
    
    'Front Seat, Right Side',
    'Front Seat, Middle',
    'Front Seat, Unknown',
    'Front Seat, Other', 
    'Not Reported',
    'Reported as Unknown', 
    
    'Second Seat, Left Side', 
    'Second Seat, Middle',
    'Second Seat, Right Side', 
    'Second Seat, Unknown', 
    'Second Seat, Other',
    
    'Third Seat, Left Side',
    'Third Seat, Middle', 
    'Third Seat, Right Side', 
    'Third Seat, Unknown', 
    'Third Seat, Other',
    'Fourth Seat, Left Side',
    'Fourth Seat, Right Side', 
    'Fourth Seat, Middle',
    'Fourth Seat, Other',
    
    'Riding on Exterior of Vehicle', 
    'Appended to a Motor Vehicle for Motion',
    
    'Other Passenger in enclosed passenger or cargo area',
    'Sleeper Section of Cab (Truck)',
    'Other Passenger in passenger or cargo area, unknown whether or not enclosed',
    'Other Passenger in unenclosed passenger or cargo area',
    
    'Trailing Unit',
    
],[
    'Driver',
    
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    'Front_passenger',
    
    'Second_row',
    'Second_row',
    'Second_row',
    'Second_row',
    'Second_row',
    
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    'Third_or_4 Row',
    
    'Riding_outside',
    'Riding_outside',
    
    'Cargo_area',
    'Cargo_area',
    'Cargo_area',
    'Cargo_area',
    
    'Trailer'    
])
avp_df['seat_imname'].unique()

In [ ]:
avp_df['age_im_temp']=avp_df['age_im'].replace([997,998,999],[np.NaN,np.NaN,np.NaN])
avp_df['age_im_temp'].describe()

In [ ]:
# Recode age - impute median age = 34 years for missing
avp_df['age_im']=avp_df['age_im'].replace([998,999],[34,34])
avp_df['age_im'].describe()

In [ ]:
# Write to CSV SEMI CLEAN - NEEDS SOME FIELDS DROPPED - NOT THE FINAL SET FOR MACHINE LEARNING
avp_df.to_csv('all_gas_no_brakes_semiclean.csv')

# NOT UPDATED BELOW THIS POINT

# FINAL PROJECT MACHINE LEARNING CODE


In [ ]:
# CONNECT DATA TO DATABASE
crash_df = pd.read_csv("Resources/crash.csv")
crash_df.head()

In [ ]:
# PREPROCESS THE DATA

In [ ]:
crash_df.columns

In [ ]:
# Check unique catagorical 
crash_df.nunique()

In [ ]:
# Look at XXXX value counts for binning
XXXX_counts = application_df.XXXX.value_counts()
#  How many XXXX counts are greater than X?
XXXX_counts[XXXX_counts>X]

In [ ]:
#  How many XXXX counts are less than or equal to X?
XXXX_counts[XXXX_counts <= X]

In [ ]:
# Visualize the value counts of XXXX
XXXX_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than or equal to X.
replace_crash = list(XXXX_counts[XXXX_counts <= X].index)

# Replace in dataframe
for crash in replace_crash:
    crash_df.XXXX = crash_df.XXXX.replace(app,"Other")
    
# Check to make sure binning was successful
crash_df.XXXX.value_counts()

In [ ]:
# Generate our categorical variable lists
crash_cat = crash_df.dtypes[crash_df.dtypes == "object"].index.tolist()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(crash_df[crash_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(crash_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
crash_df = crash_df.merge(encode_df,left_index=True, right_index=True)
crash_df = crash_df.drop(crash_cat,1)
crash_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = crash_df["YVALUE"].values
X = crash_df.drop(["YVALUE"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("crash_checkpoints/",exist_ok=True)
checkpoint_path = "crash_checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch')

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save("Crash.h5")